## Importing the Dependencies

In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

## Load Cleaned CSVs

In [ ]:
benin = pd.read_csv('data/benin-malanville.csv')
sierra = pd.read_csv('data/sierraleone-bumbuna.csv')
togo = pd.read_csv('data/togo-dapaong_qc.csv')

benin['Country'] = 'Benin'
sierra['Country'] = 'Sierra Leone'
togo['Country'] = 'Togo'

df_all = pd.concat([benin, sierra, togo])


## Metric Comparison

In [ ]:
sns.boxplot(x='Country', y='GHI', data=df_all)
plt.title('GHI Comparison')
plt.show()

# Summary Table
summary = df_all.groupby('Country')[['GHI','DNI','DHI']].agg(['mean','median','std'])
print(summary)


## Statistical Testing

In [ ]:
from scipy.stats import f_oneway

f_stat, p_val = f_oneway(
    benin['GHI'], sierra['GHI'], togo['GHI']
)
print(f"ANOVA p-value: {p_val}")


## Bar chart ranking countries

In [ ]:
df_all.groupby('Country')['GHI'].mean().sort_values().plot(kind='bar')
plt.title('Average GHI by Country')
plt.show()
